In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import seaborn as sns

In [ ]:
df=pd.read_csv("../input/CreditCardUsage.csv")

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.describe().T

In [ ]:
#Remove Unneccasary column
df.drop('CUST_ID', axis = 1, inplace = True)

In [ ]:
sns.heatmap(df.corr(), xticklabels=df.columns, yticklabels=df.columns)

In [ ]:

import matplotlib.pyplot as plt


In [ ]:
missing = df.isna().sum()
print(missing)

Since the number of missing values is low (the total number of samples is 8950), we'll impute with median of the columns.

We'll then use the elbow method to find a good number of clusters with the KMeans++ algorithm

    WCSS always decreases with the increase in the number of clusters. However, it should be noted that, the rate of drop in WCSS starts to drop as we increase the number of clusters. This would be our hint. We need to stop at the number of clusters from where the rate of drop in WCSS doesn’t drop substantially (in other words, the rate of drop is very less). This is sometimes also termed as Elbow method.

In [ ]:
df = df.fillna( df.median() )
#We use standardScaler() to normalize our dataset.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Scaled_df = scaler.fit_transform(df)
df_scaled = pd.DataFrame(Scaled_df,columns=df.columns)
df_scaled.head()

In [ ]:
#df = df.fillna( df.median() )
# Let's assume we use all cols except CustomerID
vals = df_scaled.iloc[ :, :].values
from sklearn.cluster import KMeans
# Use the Elbow method to find a good number of clusters using WCSS

wcss = []
for i in range( 1, 30 ):
    kmeans = KMeans(n_clusters=i, init="k-means++", n_init=10, max_iter=300) 
    kmeans.fit_predict( vals )
    wcss.append( kmeans.inertia_ )
   
plt.plot( wcss, 'ro-', label="WCSS")
plt.title("Computing WCSS for KMeans++")
plt.xlabel("Number of clusters")
plt.ylabel("WCSS")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=8, init="k-means++", n_init=10, max_iter=300) 
y_pred = kmeans.fit_predict( vals )
labels = kmeans.labels_
df_scaled["Clus_km"] = labels

# As it's difficult to visualise clusters when the data is high-dimensional - we'll see
# if Seaborn's pairplot can help us see how the clusters are separating out the samples.   
import seaborn as sns
df_scaled["cluster"] = y_pred
cols = list(df_scaled.columns)


sns.lmplot(data=df_scaled,x='BALANCE',y='PURCHASES',hue='Clus_km')



#plt.scatter(X[:,0], X[:,2], c=labels.astype(np.float), alpha=0.5)
#plt.xlabel('BALANCE', fontsize=18)
#plt.ylabel('PURCHASES', fontsize=16)

#sns.pairplot( df[ cols ], hue="cluster")

    Let's choose n=8 clusters. As it's difficult to visualize clusters when we have more than 2-dimensions, we'll see if Seaborn's pairplot can show how the clusters are segmenting the samples.

In [ ]:
#using best cols  :

best_cols = ["BALANCE","PURCHASES","CASH_ADVANCE","CREDIT_LIMIT","PAYMENTS","MINIMUM_PAYMENTS"]
kmeans = KMeans(n_clusters=8, init="k-means++", n_init=10, max_iter=300) 
best_vals = df_scaled[best_cols].iloc[ :, :].values
y_pred = kmeans.fit_predict( best_vals )
wcss = []
for i in range( 1, 30 ):
    kmeans = KMeans(n_clusters=i, init="k-means++", n_init=10, max_iter=300) 
    kmeans.fit_predict( best_vals )
    wcss.append( kmeans.inertia_ )



In [ ]:
sns.set_palette('Set2')
sns.scatterplot(df_scaled['BALANCE'],df_scaled['PURCHASES'],hue=labels,palette='Set1')

The goal was to segment the customers in order to define a marketing strategy. Unfortunately the colors of the plots change when this kernel is rerun - but here are some thoughts:

Big Spenders with large Payments 
-------------------------------
        - they make expensive purchases and have a credit limit that is between average and high. This is only a small group of customers.

Cash Advances with large Payments - 
-------------
        - this group takes the most cash advances. They make large payments, but this appears to be a small group of customers.

Medium Spenders with third highest Payments - 
-----------
        - the second highest Purchases group (after the Big Spenders).

Highest Credit Limit but Frugal - 
---------
        - this group doesn't make a lot of purchases. It looks like the 3rd largest group of customers.
Cash Advances with Small Payments - this group likes taking cash advances, but make only small payments.

Small Spenders and Low Credit Limit -
-------
        - they have the smallest Balances after the Smallest Spenders, their Credit Limit is in the bottom 3 groups, the second largest group of customers.

Smallest Spenders and Lowest Credit Limit 
--------
        - this is the group with the lowest credit limit but they don't appear to buy much. Unfortunately this appears to be the largest group of customers.

Highest Min Payments
        --------
                - this group has the highest minimum payments (which presumably refers to "Min Payment Due" on the monthly statement. This might be a reflection of the fact that they have the second lowest Credit Limit of the groups, so it looks like the bank has identified them as higher risk.)

    So a marketing strategy that targeted the first five groups might be effective.